### Transform

Here we weill transform the data by cleaning it into a suitable format for analysis

In [39]:
import pandas as pd
import numpy as np

In [40]:
# importing the unclean data 
cases = pd.read_csv('/home/warren/covid19_ETL_project/data_library/confirmed_cases_global.csv')
deaths = pd.read_csv('/home/warren/covid19_ETL_project/data_library/confirmed_deaths_global.csv')
recovered = pd.read_csv('/home/warren/covid19_ETL_project/data_library/confirmed_recovered_global.csv')

Now that all the data has been imported, lets perform some EDA to understand more about the data first before transforming it.

We'll start with the cases data that has information about hthe number of cases recorded in each country on each day

In [41]:
cases.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,209322,209340,209358,209362,209369,209390,209406,209436,209451,209451
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,334391,334408,334408,334427,334427,334427,334427,334427,334443,334457
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,271441,271448,271463,271469,271469,271477,271477,271490,271494,271496
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,47866,47875,47875,47875,47875,47875,47875,47875,47890,47890
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,105255,105277,105277,105277,105277,105277,105277,105277,105288,105288


Lets take a look at its structure first

In [42]:
cases.shape

(289, 1147)

In [43]:
cases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 289 entries, 0 to 288
Columns: 1147 entries, Province/State to 3/9/23
dtypes: float64(2), int64(1143), object(2)
memory usage: 2.5+ MB


This DataFrame has 289 entries with 1147 columns

Taking a look at the columns

In [44]:
cases.columns

Index(['Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20', '1/23/20',
       '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       ...
       '2/28/23', '3/1/23', '3/2/23', '3/3/23', '3/4/23', '3/5/23', '3/6/23',
       '3/7/23', '3/8/23', '3/9/23'],
      dtype='object', length=1147)

The columns constitute of the following:
- `Province/State`: The province or state within the country.
- `Country/Region`: The country or region where the cases are reported.
- `Lat`: Latitude of the location.
- `Long`: Longitude of the location.
- `1/22/20`, `1/23/20`, ..., `3/9/23`: Daily confirmed cases for each date from January 22, 2020, to March 9, 2023.

Lets melt this Dataframe first and have Country/Region, Lat and Long as the id variables and the date as the variable name and the number of cases as the value name.

In [45]:
# Reshaping the data to have a long format
cases = cases.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'],
                    var_name='Date',
                    value_name='confirmed_cases')
cases.head()

,Province/State,Country/Region,Lat,Long,Date,confirmed_cases
0,NaN,Afghanistan,33.93911,67.709953,1/22/20,0
1,NaN,Albania,41.15330,20.168300,1/22/20,0
2,NaN,Algeria,28.03390,1.659600,1/22/20,0
3,NaN,Andorra,42.50630,1.521800,1/22/20,0
4,NaN,Angola,-11.20270,17.873900,1/22/20,0


Lets check for the count of missing values 

In [46]:
cases.isnull().sum()

Province/State     226314
Country/Region          0
Lat                  2286
Long                 2286
Date                    0
confirmed_cases         0
dtype: int64

checking percentage of missing values

In [47]:
cases.isnull().mean() * 100

Province/State     68.512111
Country/Region      0.000000
Lat                 0.692042
Long                0.692042
Date                0.000000
confirmed_cases     0.000000
dtype: float64

From the data the 'province/state' column has missing values of more than 50% so we will drop it.

Since we only have less than 1% values missing in the latitute and longitude columns, we can fill them with the mode since they are geographical coordinates and we can assume the most common values are representative

In [48]:
# dropping the 'Province/State' column as it has more than 50% missing values
cases = cases.drop(columns=['Province/State'], axis=1)
cases.head()

,Country/Region,Lat,Long,Date,confirmed_cases
0,Afghanistan,33.93911,67.709953,1/22/20,0
1,Albania,41.15330,20.168300,1/22/20,0
2,Algeria,28.03390,1.659600,1/22/20,0
3,Andorra,42.50630,1.521800,1/22/20,0
4,Angola,-11.20270,17.873900,1/22/20,0


In [49]:
# filling missing values with the mode for latitude and longitude
cases['Lat'] = cases['Lat'].fillna(cases['Lat'].mode()[0])
cases['Long'] = cases['Long'].fillna(cases['Long'].mode()[0])

# lets check again for missing values
cases.isnull().sum()

Country/Region     0
Lat                0
Long               0
Date               0
confirmed_cases    0
dtype: int64

Lets rename the 'country/region' column to country

In [50]:
cases = cases.rename(columns={'Country/Region': 'country'})
cases.head()

,country,Lat,Long,Date,confirmed_cases
0,Afghanistan,33.93911,67.709953,1/22/20,0
1,Albania,41.15330,20.168300,1/22/20,0
2,Algeria,28.03390,1.659600,1/22/20,0
3,Andorra,42.50630,1.521800,1/22/20,0
4,Angola,-11.20270,17.873900,1/22/20,0


Lets also convert the date column to Datatime format and sort the data by country and date

In [51]:
# converting date to datetime format
cases['Date'] = pd.to_datetime(cases['Date'])
# sorting the data by country and date
cases = cases.sort_values(by=['country', 'Date']).reset_index(drop=True)
cases.head()

/tmp/ipykernel_20384/3446920370.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cases['Date'] = pd.to_datetime(cases['Date'])


,country,Lat,Long,Date,confirmed_cases
0,Afghanistan,33.93911,67.709953,2020-01-22,0
1,Afghanistan,33.93911,67.709953,2020-01-23,0
2,Afghanistan,33.93911,67.709953,2020-01-24,0
3,Afghanistan,33.93911,67.709953,2020-01-25,0
4,Afghanistan,33.93911,67.709953,2020-01-26,0


Lets use a consistent format in column names by converting them to lowercase and replacing spaces with underscores 

In [52]:
cases.columns = cases.columns.str.lower().str.replace(' ', '_')
cases.head()

,country,lat,long,date,confirmed_cases
0,Afghanistan,33.93911,67.709953,2020-01-22,0
1,Afghanistan,33.93911,67.709953,2020-01-23,0
2,Afghanistan,33.93911,67.709953,2020-01-24,0
3,Afghanistan,33.93911,67.709953,2020-01-25,0
4,Afghanistan,33.93911,67.709953,2020-01-26,0


In [53]:
cases.dtypes

country                    object
lat                       float64
long                      float64
date               datetime64[ns]
confirmed_cases             int64
dtype: object

we can see that the datatypes are correct, the date is in datetime format, the country is in string format, and the confirmed cases are in integer format. The latitude and longitude are in float format.

We can now proceed to the next step of the ETL process which is to load it into the database.


We can now save this cleaned data to a new CSV file

In [54]:
with open('/home/warren/covid19_ETL_project/data_library/cleaned_confirmed_cases_global.csv', 'w') as f:
    cases.to_csv(f, index=False)

Now lets perform the same transformations on the deaths dataset

In [55]:
deaths.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,7896,7896,7896,7896,7896,7896,7896,7896,7896,7896
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3598,3598,3598,3598,3598,3598,3598,3598,3598,3598
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,6881,6881,6881,6881,6881,6881,6881,6881,6881,6881
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,165,165,165,165,165,165,165,165,165,165
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,1933,1933,1933,1933,1933,1933,1933,1933,1933,1933


Lets check its structure

In [56]:
deaths.shape

(289, 1147)

In [57]:
deaths.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 289 entries, 0 to 288
Columns: 1147 entries, Province/State to 3/9/23
dtypes: float64(2), int64(1143), object(2)
memory usage: 2.5+ MB


The DataFrame has 289 entries and 1147 columns

Lets take a look at these columns

In [58]:
deaths.columns

Index(['Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20', '1/23/20',
       '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       ...
       '2/28/23', '3/1/23', '3/2/23', '3/3/23', '3/4/23', '3/5/23', '3/6/23',
       '3/7/23', '3/8/23', '3/9/23'],
      dtype='object', length=1147)

The columns constitute of the following:
- `Province/State`: The province or state within the country.
- `Country/Region`: The country or region where the cases are reported.
- `Lat`: Latitude of the location.
- `Long`: Longitude of the location.
- `1/22/20`, `1/23/20`, ..., `3/9/23`: Daily confirmed deaths for each date from January 22, 2020, to March 9, 2023.

Lets melt this Dataframe first and have Country/Region, Lat and Long as the id variables and the date as the variable name and the number of cases as the value name.

In [59]:
deaths = deaths.melt(id_vars = ['Province/State', 'Country/Region', 'Lat', 'Long'],
                    var_name='Date',
                    value_name='confirmed_deaths')
deaths.head()

,Province/State,Country/Region,Lat,Long,Date,confirmed_deaths
0,NaN,Afghanistan,33.93911,67.709953,1/22/20,0
1,NaN,Albania,41.15330,20.168300,1/22/20,0
2,NaN,Algeria,28.03390,1.659600,1/22/20,0
3,NaN,Andorra,42.50630,1.521800,1/22/20,0
4,NaN,Angola,-11.20270,17.873900,1/22/20,0


Lets check for missing values

In [60]:
deaths.isnull().sum()

Province/State      226314
Country/Region           0
Lat                   2286
Long                  2286
Date                     0
confirmed_deaths         0
dtype: int64

Lets look at the percentage of missing values in the dataset

In [61]:
deaths.isnull().mean() * 100

Province/State      68.512111
Country/Region       0.000000
Lat                  0.692042
Long                 0.692042
Date                 0.000000
confirmed_deaths     0.000000
dtype: float64

The percentage of missing values in the dataset is quite high, especially for the 'Province/State' column. We will drop this column as it has more than 50% missing values.

For the 'Lat' and 'Long' columns we'll fill in the missing values with the mode of their respective columns since the missing values are less than 1%

In [62]:
deaths.drop(columns = ['Province/State'], axis=1, inplace=True)
deaths['Lat'] = deaths['Lat'].fillna(deaths['Lat'].mode()[0])
deaths['Long'] = deaths['Long'].fillna(deaths['Long'].mode()[0])
# checking for missing values again
deaths.isnull().sum()

Country/Region      0
Lat                 0
Long                0
Date                0
confirmed_deaths    0
dtype: int64

Lets once again rename the 'Country/Region' column to country 

In [63]:
deaths = deaths.rename(columns={'Country/Region': 'country'})
deaths.head()

,country,Lat,Long,Date,confirmed_deaths
0,Afghanistan,33.93911,67.709953,1/22/20,0
1,Albania,41.15330,20.168300,1/22/20,0
2,Algeria,28.03390,1.659600,1/22/20,0
3,Andorra,42.50630,1.521800,1/22/20,0
4,Angola,-11.20270,17.873900,1/22/20,0


Also, lets convert the date column to date_time and sort the data according t country and date 

In [64]:
deaths['Date'] = pd.to_datetime(deaths['Date'])
# sorting the data by country and date
deaths = deaths.sort_values(by=['country', 'Date']).reset_index(drop=True)
deaths.head()

/tmp/ipykernel_20384/3996818340.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  deaths['Date'] = pd.to_datetime(deaths['Date'])


,country,Lat,Long,Date,confirmed_deaths
0,Afghanistan,33.93911,67.709953,2020-01-22,0
1,Afghanistan,33.93911,67.709953,2020-01-23,0
2,Afghanistan,33.93911,67.709953,2020-01-24,0
3,Afghanistan,33.93911,67.709953,2020-01-25,0
4,Afghanistan,33.93911,67.709953,2020-01-26,0


Lets use a consistent format in column names by converting them to lowercase and replacing spaces with underscores 

In [65]:
deaths.columns = deaths.columns.str.lower().str.replace(' ', '_')
deaths.head()

,country,lat,long,date,confirmed_deaths
0,Afghanistan,33.93911,67.709953,2020-01-22,0
1,Afghanistan,33.93911,67.709953,2020-01-23,0
2,Afghanistan,33.93911,67.709953,2020-01-24,0
3,Afghanistan,33.93911,67.709953,2020-01-25,0
4,Afghanistan,33.93911,67.709953,2020-01-26,0


In [66]:
deaths.dtypes

country                     object
lat                        float64
long                       float64
date                datetime64[ns]
confirmed_deaths             int64
dtype: object

we can see that the datatypes are correct, the date is in datetime format, the country is in string format, and the confirmed deaths are in integer format. The latitude and longitude are in float format.

We can now proceed to the next step of the ETL process which is to load it into the database.


In [67]:
with open('/home/warren/covid19_ETL_project/data_library/cleaned_confirmed_deaths_global.csv', 'w') as f:
    deaths.to_csv(f, index=False)

Finally, lets perform the same transformations on the recovered dataset

In [68]:
recovered.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Lets check its structure

In [69]:
recovered.shape

(274, 1147)

In [70]:
recovered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274 entries, 0 to 273
Columns: 1147 entries, Province/State to 3/9/23
dtypes: float64(2), int64(1143), object(2)
memory usage: 2.4+ MB


The dataset has 274 entries and 1147 columns

Lets take a look at the columns

In [71]:
recovered.columns

Index(['Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20', '1/23/20',
       '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       ...
       '2/28/23', '3/1/23', '3/2/23', '3/3/23', '3/4/23', '3/5/23', '3/6/23',
       '3/7/23', '3/8/23', '3/9/23'],
      dtype='object', length=1147)

The columns constitute of the following:
- `Province/State`: The province or state within the country.
- `Country/Region`: The country or region where the cases are reported.
- `Lat`: Latitude of the location.
- `Long`: Longitude of the location.
- `1/22/20`, `1/23/20`, ..., `3/9/23`: Daily confirmed recoveries for each date from January 22, 2020, to March 9, 2023.

Lets melt this Dataframe first and have Country/Region, Lat and Long as the id variables and the date as the variable name and the number of cases as the value name.

In [72]:
recovered = recovered.melt(id_vars = ['Province/State', 'Country/Region', 'Lat', 'Long'],
                           var_name='Date',
                           value_name='confirmed_recovered')
recovered.head()

,Province/State,Country/Region,Lat,Long,Date,confirmed_recovered
0,NaN,Afghanistan,33.93911,67.709953,1/22/20,0
1,NaN,Albania,41.15330,20.168300,1/22/20,0
2,NaN,Algeria,28.03390,1.659600,1/22/20,0
3,NaN,Andorra,42.50630,1.521800,1/22/20,0
4,NaN,Angola,-11.20270,17.873900,1/22/20,0


Lets check for missing values

In [73]:
recovered.isnull().sum()

Province/State         227457
Country/Region              0
Lat                      1143
Long                     1143
Date                        0
confirmed_recovered         0
dtype: int64

Looking at the percentage of missing values in the dataset

In [74]:
recovered.isnull().mean() * 100

Province/State         72.627737
Country/Region          0.000000
Lat                     0.364964
Long                    0.364964
Date                    0.000000
confirmed_recovered     0.000000
dtype: float64

The percentage of missing values in the dataset is quite high, especially for the 'Province/State' column. We will drop this column as it has more than 50% missing values.

For the 'Lat' and 'Long' columns we'll fill in the missing values with the mode of their respective columns since the missing values are less than 1%

In [75]:
recovered.drop(columns= ['Province/State'], axis=1, inplace=True)
recovered['Lat'] = recovered['Lat'].fillna(recovered['Lat'].mode()[0])
recovered['Long'] = recovered['Long'].fillna(recovered['Long'].mode()[0])
# checking for missing values again
recovered.isnull().sum()

Country/Region         0
Lat                    0
Long                   0
Date                   0
confirmed_recovered    0
dtype: int64

Also, lets convert the 'Country/State' column to 'country' and 'confirmed_recovered' to 'confirmed_recoveries'

In [76]:
recovered.rename(columns = {'Country/Region': 'country',
                            'confirmed_recovered' :'confirmed_recoveries'}, inplace=True)
recovered.head()

,country,Lat,Long,Date,confirmed_recoveries
0,Afghanistan,33.93911,67.709953,1/22/20,0
1,Albania,41.15330,20.168300,1/22/20,0
2,Algeria,28.03390,1.659600,1/22/20,0
3,Andorra,42.50630,1.521800,1/22/20,0
4,Angola,-11.20270,17.873900,1/22/20,0


Renaming the date column datatype to date time datatype and sorting the DataFrame according to country and confirmed_recoveries

In [77]:
recovered['Date'] = pd.to_datetime(recovered['Date'])
# sorting the DataFrame according to country and confirmed_recoveries
recovered = recovered.sort_values(by=['country', 'Date']).reset_index(drop=True)
recovered.head()

/tmp/ipykernel_20384/312883383.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  recovered['Date'] = pd.to_datetime(recovered['Date'])


,country,Lat,Long,Date,confirmed_recoveries
0,Afghanistan,33.93911,67.709953,2020-01-22,0
1,Afghanistan,33.93911,67.709953,2020-01-23,0
2,Afghanistan,33.93911,67.709953,2020-01-24,0
3,Afghanistan,33.93911,67.709953,2020-01-25,0
4,Afghanistan,33.93911,67.709953,2020-01-26,0


Lets use a consistent format in column names by converting them to lowercase and replacing spaces with underscores 

In [78]:
recovered.columns = recovered.columns.str.lower().str.replace(' ', '_')
recovered.head()

,country,lat,long,date,confirmed_recoveries
0,Afghanistan,33.93911,67.709953,2020-01-22,0
1,Afghanistan,33.93911,67.709953,2020-01-23,0
2,Afghanistan,33.93911,67.709953,2020-01-24,0
3,Afghanistan,33.93911,67.709953,2020-01-25,0
4,Afghanistan,33.93911,67.709953,2020-01-26,0


Saving the transforemed data into a csv

In [79]:
with open('/home/warren/covid19_ETL_project/data_library/cleaned_confirmed_recovered_global.csv', 'w') as f:
    recovered.to_csv(f, index=False)